In [2]:
from workscript_utils import *
import utils.math_utils as mu
import time
import cvxpy as cp
from exp_utils  import *

In [3]:
X,y = get_dataset("make_circles")
n_qubits= X.shape[1]





In [ ]:
def kappa(delta):
    return np.sqrt(2*np.log(1/delta))
def primal_rob_no_dummy(K_star, y_train, delta_1 = 0.1, delta_2 = 0.1, 
                        C = 1, shots = 20, circuit_type = 'gates', 
                        sv_threshold = 1e-3, numerical_shift = 1e-3): 
    g = 1
    if circuit_type == 'swap':
        g = 2
    n = K_star.shape[1]
    ##Shifting the kernel to ensure numerical stability of the solver
    K_star = K_star + numerical_shift*np.eye(n)
    Y=  y_train.reshape(n,1) 
    beta = cp.Variable((n,1))
    v = cp.Variable()
    t = cp.Variable()
    k1 = cp.Parameter(nonneg=True)
    k2 = cp.Parameter(nonneg=True)

    loss = cp.sum(cp.pos(1 + k1*t - cp.multiply(Y, K_star @ beta + v)))

    socp_constraint = [cp.SOC(t,beta)]
    reg1 = cp.quad_form(beta,K_star )
    #reg2 = cp.norm(beta, 2)
    reg2 = cp.quad_form(beta,np.eye(n))

    prob = cp.Problem(cp.Minimize(C*loss + 0.5*reg1 + 0.5*k2* reg2  ), socp_constraint)
    k1.value = kappa(delta_1)*(g/np.sqrt(shots))
    k2.value = kappa(delta_2)*(g/np.sqrt(shots))
    prob.solve()
    scaled_beta = np.abs(beta.value)/np.max(np.abs(beta.value))
    SV =  [i  for i in range(n) if scaled_beta[i] > sv_threshold ]
    return beta.value.flatten(), v.value, SV


def primal_rob_(K_star, y_train, delta_1 = 0.1, delta_2 = 0.1, 
                        C = 1, shots = 20, circuit_type = 'gates', 
                        sv_threshold = 1e-3, numerical_shift = 1e-4, 
                          momentum = 0.0, lr = 0.1): 
    g = 1
    if circuit_type == 'swap':
        g = 2
        
    n = K_star.shape[1]
    ##Shifting the kernel to ensure numerical stability of the solver
    K_star = K_star + numerical_shift*np.eye(n)
    Y=  y_train.reshape(n,1) 
    beta = 0.5*C*np.random.rand(n)
    v = np.random.rand()
    k1 = kappa(delta_1)*(g/np.sqrt(shots))
    k2 = kappa(delta_2)*(g/np.sqrt(shots))
   

    loss = np.sum(np.max(1 + k1*t - cp.multiply(Y, K_star @ beta + v)))

    socp_constraint = [cp.SOC(t,beta)]
    reg1 = cp.quad_form(beta,K_star )
    #reg2 = cp.norm(beta, 2)
    reg2 = cp.quad_form(beta,np.eye(n))

    prob = cp.Problem(cp.Minimize(C*loss + 0.5*reg1 + 0.5*k2* reg2  ), socp_constraint)

    prob.solve()
    scaled_beta = np.abs(beta.value)/np.max(np.abs(beta.value))
    SV =  [i  for i in range(n) if scaled_beta[i] > sv_threshold ]
    return beta.value.flatten(), v.value, SV

def run_rob(args):
    key, m, delta_1, delta_2, C, shots, circuit_type, training_type, trials = args

    print(f'[JOB {args}]', flush=True)
    K_train, y_train = return_kernel(key, m )
    print(f"K[10,1] = {K_train[10,1]}")
    if C  == "1/sqrt(m)":
        C = 1/np.sqrt(m)

    if C == "optimal":
        C = find_optimal_C(K_train,y_train)

    return primal_rob_no_dummy(K_trai

In [16]:
n, y_train, delta_1 = delta_1,
                            delta_2 = delta_2, shots = shots,  circuit_type = circuit_type)



In [20]:
run_rob( (("Havliscek","Generated"),20, 0.1, 0.1, 1 , 20, "swap", "exact",  10) )

[JOB (('Havliscek', 'Generated'), 20, 0.1, 0.1, 1, 20, 'swap', 'exact', 10)]


Generating data:('Havliscek', 'Generated')


Data gen finished


K[10,1] = 0.058278627680928466


ValueError: cannot reshape array of size 20 into shape (60,1)

In [5]:
node = create_device("havlicek",n_qubits=n_qubits)
K_star,K_star_test_train,y_train,y_test = evaluate_kernel(X,y,node)
K_gauss,K_gauss_test_train = evaluate_kernel(X,y,"gaussian")
print(mu.eigmin(K_star))
print(mu.eigmin(K_gauss))

[Quantum case] Time taken for evaluating training kernel: 6.785682201385498


[Quantum Case] Time taken for evaluating test-train kernel: 31.658611297607422
[Gaussian case] Total time to evaluate both train and test-train kernel: 0.13519525527954102
-1.8267256385831218e-15
8.074526725039671e-17


In [8]:
start=time.time()
beta_1,b_1,SV1 = primal_rob_no_dummy(K_star, y_train, delta_1 = 1.0, delta_2 = 1.0, C =0.5, numerical_shift = 1e-3)
#beta_2,b_2,SV2 = nominal_classifier(K_star,y_train,svm_type="dual",C=1);print("time taken:",time.time()-start);start=time.time()
beta_3,b_3,SV3 = nominal_classifier(K_star,y_train,svm_type="SVC",C=0.5);print("time taken:",time.time()-start);start=time.time()


print("beta_1:",beta_1,"b_1",b_1)
#print("beta_2:",beta_2,"b_2",b_2)
print("beta_3:",beta_3,"b_3",b_3)
print( np.linalg.norm(beta_1 - y_train*beta_3, np.inf))

time taken: 0.05143451690673828
beta_1: [-5.000e-01 -5.000e-01  5.000e-01  4.171e-05 -5.000e-01  5.000e-01
 -5.000e-01 -5.000e-01 -5.000e-01  5.000e-01 -1.429e-01 -5.000e-01
  5.000e-01 -5.000e-01  5.000e-01 -5.000e-01 -1.570e-05  5.000e-01
 -5.000e-01  5.000e-01 -5.000e-01  5.000e-01 -5.000e-01 -5.000e-01
  5.000e-01  5.000e-01  5.000e-01 -5.000e-01  5.000e-01  8.729e-05
 -5.000e-01  5.000e-01  5.000e-01  5.000e-01  5.440e-05 -5.000e-01
 -5.000e-01  5.000e-01 -5.000e-01  4.165e-01  5.000e-01 -5.000e-01
 -3.771e-01  5.000e-01 -2.593e-06  5.000e-01 -5.000e-01  5.000e-01
  5.000e-01  5.000e-01 -5.000e-01  1.675e-01  5.000e-01  5.594e-02
 -5.000e-01  3.800e-01 -5.000e-01  5.000e-01 -5.000e-01 -5.000e-01] b_1 -0.49335604923643445
beta_3: [0.5   0.5   0.5   0.    0.5   0.5   0.5   0.5   0.5   0.5   0.132 0.5
 0.5   0.5   0.5   0.5   0.    0.5   0.5   0.5   0.5   0.5   0.5   0.5
 0.5   0.5   0.5   0.5   0.5   0.    0.5   0.5   0.5   0.5   0.    0.5
 0.5   0.5   0.5   0.418 0.5   0.5   0.389 

In [83]:
np.linalg.eigh(K_gauss)

(array([1.188e-16, 4.378e-15, 1.210e-13, 2.870e-13, 1.588e-12, 1.020e-11,
        1.591e-10, 4.160e-10, 6.241e-10, 1.307e-09, 1.819e-09, 1.799e-08,
        2.928e-08, 6.700e-08, 1.346e-07, 2.573e-07, 3.586e-07, 1.031e-06,
        1.275e-06, 5.851e-06, 1.015e-05, 2.180e-05, 2.516e-05, 4.834e-05,
        6.517e-05, 1.393e-04, 2.758e-04, 3.598e-04, 6.494e-04, 1.134e-03,
        1.539e-03, 2.670e-03, 3.835e-03, 6.207e-03, 8.959e-03, 1.086e-02,
        1.507e-02, 2.949e-02, 3.689e-02, 4.114e-02, 5.658e-02, 8.142e-02,
        9.760e-02, 1.552e-01, 1.886e-01, 2.428e-01, 3.017e-01, 4.328e-01,
        5.068e-01, 5.741e-01, 6.785e-01, 1.221e+00, 1.479e+00, 2.873e+00,
        3.225e+00, 5.280e+00, 7.108e+00, 9.334e+00, 1.112e+01, 1.489e+01]),
 array([[ 0.000e+00, -1.421e-03, -2.827e-03, ..., -1.051e-02,  1.580e-01,
         -1.802e-01],
        [-3.024e-07,  4.697e-06,  2.630e-06, ..., -1.596e-02, -1.800e-01,
         -8.733e-02],
        [ 1.043e-06, -9.855e-07, -4.289e-05, ..., -2.517e-01,  1.5

In [100]:
Y =  y_train.reshape(60,1)
X = K_star 
C = 1
import cvxpy as cp
beta = cp.Variable((n,1))
v = cp.Variable()
loss = cp.sum(cp.pos(1 - cp.multiply(Y, X @ beta - v)))
#reg = cp.norm(beta, 2)
reg = cp.quad_form(beta,X)
#reg = cp.quad_form(beta,np.eye(n))
lambd = cp.Parameter(nonneg=True)
prob = cp.Problem(cp.Minimize(C*loss + 0.5*reg))
prob.solve()


16.038461060284348

In [117]:



Y =  y_train.reshape(60,1)
X = K_star 
C = 1
import cvxpy as cp
beta = cp.Variable((n,1))
v = cp.Variable()
t = cp.Variable()
#k1 = cp.Parameter(nonneg=True)
k1 = 0.1
k2 = 0.0
loss = cp.sum(cp.pos(1 + k1*t - cp.multiply(Y, X @ beta + v)))

socp_constraint = [cp.SOC(t,beta)]
#constraint = [t >= 0  ]
#reg = cp.norm(beta, 2)
reg = cp.quad_form(beta,X + k2*np.eye(n))
#reg = cp.quad_form(beta,np.eye(n))

prob = cp.Problem(cp.Minimize(C*loss + 0.5*reg), socp_constraint)
prob.solve()

18.171000876608794

In [118]:
t.value

array(0.879)

In [119]:
np.linalg.norm(beta.value)

0.8792826626399073